1. Configuration


In [0]:
dbutils.widgets.text('root_path','FileStore/')
dbutils.widgets.text('synth_in', 'FileStore/shared_uploads/omkars1202@gmail.com/')
root_path=dbutils.widgets.get('root_path')
synth_in=dbutils.widgets.get('synth_in')

In [0]:
import os
from pyspark.sql import functions as F
from pyspark.sql import Window

In [0]:
synthea_path  = synth_in
delta_root_path = f"{root_path}delta/"
print(f'Synthea Raw Path: {synthea_path}\n Delta Output Path:{delta_root_path}')

Synthea Raw Path: FileStore/shared_uploads/omkars1202@gmail.com/
 Delta Output Path:FileStore/delta/


In [0]:
print(f'Synthea Raw Path: {synthea_path}\nDelta Output Path: {delta_root_path}')

Synthea Raw Path: FileStore/shared_uploads/omkars1202@gmail.com/
Delta Output Path: FileStore/delta/


In [0]:
display(dbutils.fs.ls(synthea_path))


path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/allergies.csv,allergies.csv,70010,1708397768000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/careplans.csv,careplans.csv,672599,1708397768000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/conditions.csv,conditions.csv,1062972,1708397768000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/devices.csv,devices.csv,17498,1708397768000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/encounters.csv,encounters.csv,16474491,1708397774000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/imaging_studies.csv,imaging_studies.csv,205364,1708397769000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/immunizations.csv,immunizations.csv,2225013,1708397771000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/medications.csv,medications.csv,10884009,1708397775000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/observations.csv,observations.csv,43094924,1708397785000
dbfs:/FileStore/shared_uploads/omkars1202@gmail.com/organizations.csv,organizations.csv,169568,1708397776000


In [0]:
datasets= ['allergies',
          'careplans',
          'conditions',
          'devices',
          'encounters',
          'imaging_studies',
          'immunizations',
          'medications',
          'observations',
          'organizations',
          'patients',
          'payer_transitions',
          'payers',
          'procedures',
          'providers',
          'supplies'
         ]

2. CSV Files as Spark Dataframes

In [0]:
# create a python dictionary of dataframes
df_dict = {}
for dataset in datasets:
    df_dict[dataset] = spark.read.csv('dbfs:/{}/{}.csv'.format(synthea_path,dataset),header=True,inferSchema=True)

In [0]:
# Print the dictionary keys (file names)
print("DataFrame keys (file names):", df_dict.keys())

DataFrame keys (file names): dict_keys(['allergies', 'careplans', 'conditions', 'devices', 'encounters', 'imaging_studies', 'immunizations', 'medications', 'observations', 'organizations', 'patients', 'payer_transitions', 'payers', 'procedures', 'providers', 'supplies'])


In [0]:
import pandas as pd
dataframes=[(x[0],x[1].count()) for x in list(df_dict.items())]
display(pd.DataFrame(dataframes,columns=['dataset','n_records']).sort_values(by=['n_records'],ascending=False))

dataset,n_records
observations,299697
encounters,53346
medications,42989
procedures,34981
immunizations,15478
conditions,8376
providers,5855
payer_transitions,3801
careplans,3483
patients,1171


3. De-identify Patient PHI (Masking to be performed in the Production ETL process)

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd
import hashlib

In [0]:
def mask_pii(pii_col: pd.Series) -> pd.Series:
    '''
    mask_pii: function takes a pandas series and returned sha1 hash values of elements
    '''
    sha_value = pii_col.map(lambda x: hashlib.sha1(x.encode()).hexdigest())
    return sha_value
 
mask_pii_udf = pandas_udf(mask_pii, returnType=StringType())

We then use this function to mask pii columns for a given set of columns, namely:

['SSN','DRIVERS','PASSPORT','PREFIX','FIRST','LAST','SUFFIX','MAIDEN','BIRTHPLACE','ADDRESS'].

In [0]:
pii_cols=['SSN','DRIVERS','PASSPORT','PREFIX','FIRST','LAST','SUFFIX','MAIDEN','BIRTHPLACE','ADDRESS']
patients_obfuscated = df_dict['patients']
for c in pii_cols:
  masked_col_name = c+'_masked'
  patients_obfuscated = patients_obfuscated.withColumn(c,F.coalesce(c,F.lit('null'))).withColumn(masked_col_name,mask_pii_udf(c))

Replacing the Original records with the de-identified records


In [0]:
df_dict['patients']=patients_obfuscated.drop(*pii_cols)

4. Writing Tables to Dellta Lake

In [0]:
try:
  dbutils.fs.ls(delta_root_path)
except:
  print(f'Path {delta_root_path} does not exist, creating path {delta_root_path}')
  dbutils.fs.mkdirs(delta_root_path)
print(f'Delta tables will be stored in {delta_root_path}')

Delta tables will be stored in FileStore/delta/


In [0]:
for table_name in datasets:
  table_path = f'dbfs:/FileStore/delta'+ '/bronze/{}'.format(table_name)
  df_dict[table_name].write.format('delta').mode("overwrite").save(table_path)

In [0]:
display(dbutils.fs.ls(f'{delta_root_path}/bronze/'))

path,name,size,modificationTime
dbfs:/FileStore/delta/bronze/allergies/,allergies/,0,0
dbfs:/FileStore/delta/bronze/careplans/,careplans/,0,0
dbfs:/FileStore/delta/bronze/conditions/,conditions/,0,0
dbfs:/FileStore/delta/bronze/devices/,devices/,0,0
dbfs:/FileStore/delta/bronze/encounters/,encounters/,0,0
dbfs:/FileStore/delta/bronze/imaging_studies/,imaging_studies/,0,0
dbfs:/FileStore/delta/bronze/immunizations/,immunizations/,0,0
dbfs:/FileStore/delta/bronze/medications/,medications/,0,0
dbfs:/FileStore/delta/bronze/observations/,observations/,0,0
dbfs:/FileStore/delta/bronze/organizations/,organizations/,0,0


Loading Data from Bronze Layer

In [0]:
%python
delta_root_path = f"{root_path}delta/"
print(f"Reading synthea delta tables from {delta_root_path}bronze")

Reading synthea delta tables from FileStore/delta/bronze


In [0]:
%python
import pandas as pd

table_names =['allergies','careplans','conditions','devices','encounters','imaging_studies',\
              'immunizations','medications','observations','organizations','patients',\
              'payer_transitions','payers','procedures','providers','supplies']
              
dataframes=[]
for table_name in table_names:
  df = spark.read.format('delta').load("{}/bronze/{}".format(delta_root_path,table_name))
  df.createOrReplaceTempView(table_name)
  dataframes+=[[table_name,df.count()]]

# Display number of records in each table
pdf=pd.DataFrame(dataframes,columns=['table_name','n_records'])
display(pdf.sort_values(by='n_records',ascending=False))

table_name,n_records
observations,299697
encounters,53346
medications,42989
procedures,34981
immunizations,15478
conditions,8376
providers,5855
payer_transitions,3801
careplans,3483
patients,1171


Creating Temporary views

1. Emergency Visits

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ER_VISITS AS
SELECT
  T2.encounter_id,
  T2.patient,
  T2.encounterclass,
  T2.VISIT_START_DATE,
  T2.VISIT_END_DATE
FROM
  (
    SELECT
      MIN(encounter_id) AS encounter_id,
      patient,
      encounterclass,
      VISIT_START_DATE,
      MAX(VISIT_END_DATE) AS VISIT_END_DATE
    FROM
      (
        SELECT
          CL1.id AS encounter_id,
          CL1.patient,
          CL1.encounterclass,
          CL1.start AS VISIT_START_DATE,
          CL2.stop AS VISIT_END_DATE
        FROM
          encounters CL1
          INNER JOIN encounters CL2 ON CL1.patient = CL2.patient
          AND CL1.start = CL2.start
          AND CL1.encounterclass = CL2.encounterclass
        WHERE
          CL1.encounterclass in ('emergency', 'urgent')
      ) T1
    GROUP BY
      patient,
      encounterclass,
      VISIT_START_DATE
  ) T2;

In [0]:
%sql
SELECT * FROM ER_VISITS LIMIT 100;

encounter_id,patient,encounterclass,VISIT_START_DATE,VISIT_END_DATE
0e15ba84-8842-4dc0-a7e0-c1a3321ea9f1,0047123f-12e7-486c-82df-53b3a450e365,emergency,2013-01-09T00:15:54.000+0000,2013-01-09T01:15:54.000+0000
707985cb-aef1-415b-a019-6da47f2f03e0,0149d553-f571-4e99-867e-fcb9625d07c2,emergency,2011-04-21T13:13:20.000+0000,2011-04-21T14:43:20.000+0000
8372ee4e-7a34-43b6-9e41-45267059d76f,0149d553-f571-4e99-867e-fcb9625d07c2,emergency,2013-03-15T13:13:20.000+0000,2013-03-15T14:13:20.000+0000
988ddf8a-c6b6-4e6b-8d73-62f7309aa136,01e1f394-7219-4189-bceb-3cbd90cff90b,emergency,2010-10-15T14:39:24.000+0000,2010-10-15T16:09:24.000+0000
63f652b4-a23f-40c2-a1aa-3097acba7324,01e1f394-7219-4189-bceb-3cbd90cff90b,emergency,2011-04-13T14:39:24.000+0000,2011-04-13T15:39:24.000+0000
a2bb5ad7-c196-4a80-816b-3a78aff4d44e,01e1f394-7219-4189-bceb-3cbd90cff90b,emergency,2013-07-19T14:39:24.000+0000,2013-07-19T16:09:24.000+0000
14330854-ab2a-4b3f-97cb-a93d9a792a9b,01e1f394-7219-4189-bceb-3cbd90cff90b,emergency,2013-12-27T14:39:24.000+0000,2013-12-27T16:09:24.000+0000
326542d9-f7f8-4f0b-adcf-6e07f4ae7379,0288abb6-633c-40c3-ba0c-66c7d957727e,emergency,2011-11-28T04:07:36.000+0000,2011-11-28T05:07:36.000+0000
6df6f810-d81e-4fa5-8945-f3963e6837cf,0288abb6-633c-40c3-ba0c-66c7d957727e,emergency,2012-07-04T04:07:36.000+0000,2012-07-04T05:07:36.000+0000
6e18b023-32ea-40f8-a4b8-39a5058064bf,0288abb6-633c-40c3-ba0c-66c7d957727e,emergency,2017-07-29T04:07:36.000+0000,2017-07-29T05:40:36.000+0000
